In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 30 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.


# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = '/content/drive/My Drive/Forest_TP/' # to run with the sample dataset, just set to 'hymenoptera_data'

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 0 # set to 1 if want to run on gpu.


# SETTINGS FOR DISPLAYING ON TENSORBOARD
USE_TENSORBOARD = 0 #if you want to use tensorboard set this to 1.
EXP_NAME = "fine_tuning_experiment" # if using tensorboard, enter name of experiment you want it to be displayed as.

In [ ]:
## If you want to keep a track of your network on tensorboard, set USE_TENSORBOARD TO 1 in config file.

if USE_TENSORBOARD:
    from pycrayon import CrayonClient
    cc = CrayonClient(hostname=TENSORBOARD_SERVER)
    try:
        cc.remove_experiment(EXP_NAME)
    except:
        pass
    foo = cc.create_experiment(EXP_NAME)


In [ ]:
## If you want to use the GPU, set GPU_MODE TO 1 in config file

use_gpu = GPU_MODE
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

count=0


In [ ]:
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            running_loss = 0.0
            running_corrects = 0

            counter=0
            # Iterate over data.
            for data in dset_loaders[phase]:
                inputs, labels = data
                print(inputs.size())
                # wrap them in Variable
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()),                             
                        Variable(labels.long().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                
                loss = criterion(outputs, labels)
                # print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                # if counter%10==0:
                #     print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    # print('loss backward')
                    loss.backward()
                    # print('done loss backward')
                    optimizer.step()
                    # print('done optim')
                # print evaluation statistics
                try:
                    # running_loss += loss.data[0]
                    running_loss += loss.item()
                    # print(labels.data)
                    # print(preds)
                    running_corrects += torch.sum(preds == labels.data)
                    # print('running correct =',running_corrects)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            print('trying epoch loss')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            # deep copy the model
            if phase == 'val':
                if USE_TENSORBOARD:
                    foo.add_scalar_value('epoch_loss',epoch_loss,step=epoch)
                    foo.add_scalar_value('epoch_acc',epoch_acc,step=epoch)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

In [ ]:
# This function changes the learning rate over the training model.
def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, NUM_CLASSES)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
criterion = nn.CrossEntropyLoss()

if use_gpu:
    criterion.cuda()
    model_ft.cuda()

optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=0.0001)


### Size : 32 x 32 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(32),
        transforms.RandomResizedCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)

Epoch 0/99
----------
LR is set to 0.001


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
trying epoch loss
train Loss: 0.0637 Acc: 0.6667
torch.Size([8, 3, 32, 32])
trying epoch loss
val Loss: 0.0908 Acc: 0.5000
new best accuracy =  0.5
Epoch 1/99
----------
torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
trying epoch loss
train Loss: 0.1272 Acc: 0.6667
torch.Size([8, 3, 32, 32])
trying epoch loss
val Loss: 0.0908 Acc: 0.5000
Epoch 2/99
----------
torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
trying epoch loss
train Loss: 0.1823 Acc: 0.5000
torch.Size([8, 3, 32, 32])
trying epoch loss
val Loss: 0.0909 Acc: 0.5000
Epoch 3/99
----------
torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
trying epoch loss
train Loss: 0.2410 Acc: 0.5000
torch.Size([8, 3, 32, 32])
trying epoch loss
val Loss: 0.0910 Acc: 0.5000
Epoch 4/99
----------
torch.Size([10, 3, 32, 32])
torch.Size([2, 3, 32, 32])
trying epoch loss
train Loss: 0.0646 Acc: 0.6667
torch.Size([8, 3, 32, 32])
trying epoch loss
val Loss: 0.0912 Acc: 0.50

### Size : 64x64 : 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(64),
        transforms.RandomResizedCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)

Epoch 0/99
----------
LR is set to 0.001


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 64, 64])
torch.Size([2, 3, 64, 64])
trying epoch loss
train Loss: 0.0678 Acc: 0.6667
torch.Size([8, 3, 64, 64])
trying epoch loss
val Loss: 0.0790 Acc: 0.6250
new best accuracy =  0.625
Epoch 1/99
----------
torch.Size([10, 3, 64, 64])
torch.Size([2, 3, 64, 64])
trying epoch loss
train Loss: 0.2126 Acc: 0.5000
torch.Size([8, 3, 64, 64])
trying epoch loss
val Loss: 0.0832 Acc: 0.6250
Epoch 2/99
----------
torch.Size([10, 3, 64, 64])
torch.Size([2, 3, 64, 64])
trying epoch loss
train Loss: 0.0669 Acc: 0.6667
torch.Size([8, 3, 64, 64])
trying epoch loss
val Loss: 0.0916 Acc: 0.5000
Epoch 3/99
----------
torch.Size([10, 3, 64, 64])
torch.Size([2, 3, 64, 64])
trying epoch loss
train Loss: 0.0644 Acc: 0.6667
torch.Size([8, 3, 64, 64])
trying epoch loss
val Loss: 0.0924 Acc: 0.5000
Epoch 4/99
----------
torch.Size([10, 3, 64, 64])
torch.Size([2, 3, 64, 64])
trying epoch loss
train Loss: 0.0675 Acc: 0.6667
torch.Size([8, 3, 64, 64])
trying epoch loss
val Loss: 0.0928 Acc: 0.

### Size : 128x 128

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(128),
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes


In [ ]:
# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)

Epoch 0/99
----------
LR is set to 0.001


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 128, 128])
torch.Size([2, 3, 128, 128])
trying epoch loss
train Loss: 0.0837 Acc: 0.5833
torch.Size([8, 3, 128, 128])
trying epoch loss
val Loss: 999.0296 Acc: 0.5000
new best accuracy =  0.5
Epoch 1/99
----------
torch.Size([10, 3, 128, 128])
torch.Size([2, 3, 128, 128])
trying epoch loss
train Loss: 0.1701 Acc: 0.8333
torch.Size([8, 3, 128, 128])
trying epoch loss
val Loss: 506395.0938 Acc: 0.5000
Epoch 2/99
----------
torch.Size([10, 3, 128, 128])
torch.Size([2, 3, 128, 128])
trying epoch loss
train Loss: 0.0948 Acc: 0.7500
torch.Size([8, 3, 128, 128])
trying epoch loss
val Loss: 20748.8242 Acc: 0.5000
Epoch 3/99
----------
torch.Size([10, 3, 128, 128])
torch.Size([2, 3, 128, 128])
trying epoch loss
train Loss: 0.2998 Acc: 0.5833
torch.Size([8, 3, 128, 128])
trying epoch loss
val Loss: 52736.7383 Acc: 0.5000
Epoch 4/99
----------
torch.Size([10, 3, 128, 128])
torch.Size([2, 3, 128, 128])
trying epoch loss
train Loss: 0.1081 Acc: 0.5833
torch.Size([8, 3, 128, 128])

### SIZE : 256x256 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

In [ ]:
# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)


Epoch 0/99
----------
LR is set to 0.001


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 224, 224])
torch.Size([2, 3, 224, 224])
trying epoch loss
train Loss: 0.6985 Acc: 0.4167
torch.Size([8, 3, 224, 224])
trying epoch loss
val Loss: 108.1266 Acc: 0.5000
new best accuracy =  0.5
Epoch 1/99
----------
torch.Size([10, 3, 224, 224])
torch.Size([2, 3, 224, 224])
trying epoch loss
train Loss: 0.4533 Acc: 0.5833
torch.Size([8, 3, 224, 224])
trying epoch loss
val Loss: 138908.5625 Acc: 0.5000
Epoch 2/99
----------
torch.Size([10, 3, 224, 224])
torch.Size([2, 3, 224, 224])
trying epoch loss
train Loss: 0.5686 Acc: 0.5000
torch.Size([8, 3, 224, 224])
trying epoch loss
val Loss: 3.5888 Acc: 0.5000
Epoch 3/99
----------
torch.Size([10, 3, 224, 224])
torch.Size([2, 3, 224, 224])
trying epoch loss
train Loss: 0.1376 Acc: 0.5833
torch.Size([8, 3, 224, 224])
trying epoch loss
val Loss: 175.2883 Acc: 0.5000
Epoch 4/99
----------
torch.Size([10, 3, 224, 224])
torch.Size([2, 3, 224, 224])
trying epoch loss
train Loss: 0.0932 Acc: 0.7500
torch.Size([8, 3, 224, 224])
tryin

In [ ]:
# Save model
torch.save(model_ft.state_dict(), '/content/drive/My Drive/Forest_TP/fine_tuned_best_model.pt')
#model_ft.save_state_dict('fine_tuned_best_model.pt')

### Size 512x512 : 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(512),
        transforms.RandomResizedCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes
      

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)

Epoch 0/99
----------
LR is set to 0.001


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 25 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 512, 512])
torch.Size([2, 3, 512, 512])
trying epoch loss
train Loss: 0.0554 Acc: 0.5833
torch.Size([8, 3, 512, 512])
trying epoch loss
val Loss: 12052.4492 Acc: 0.5000
new best accuracy =  0.5
Epoch 1/99
----------
torch.Size([10, 3, 512, 512])
torch.Size([2, 3, 512, 512])
trying epoch loss
train Loss: 0.0960 Acc: 0.6667
torch.Size([8, 3, 512, 512])
trying epoch loss
val Loss: 16566.1211 Acc: 0.5000
Epoch 2/99
----------
torch.Size([10, 3, 512, 512])
torch.Size([2, 3, 512, 512])
trying epoch loss
train Loss: 0.5158 Acc: 0.5000
torch.Size([8, 3, 512, 512])
trying epoch loss
val Loss: 792.0717 Acc: 0.5000
Epoch 3/99
----------
torch.Size([10, 3, 512, 512])
torch.Size([2, 3, 512, 512])
trying epoch loss
train Loss: 0.2854 Acc: 0.3333
torch.Size([8, 3, 512, 512])
trying epoch loss
val Loss: 286.4230 Acc: 0.5000
Epoch 4/99
----------
torch.Size([10, 3, 512, 512])
torch.Size([2, 3, 512, 512])
trying epoch loss
train Loss: 0.2452 Acc: 0.5000
torch.Size([8, 3, 512, 512])
tr